In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Lib

In [2]:
import glob, sys
import numpy as np
from matplotlib import rc
from astropy.table import Table
from itertools import chain
from astropy.io import fits
from astropy.io.votable import parse
import pandas as pd
from astropy.coordinates import Angle , SkyCoord
from astropy import units as u
import seaborn as sns
import scipy as sp 
from matplotlib.patches import Ellipse
from matplotlib.collections import EllipseCollection
from scipy import interpolate

import warnings
warnings.simplefilter('ignore')
!pip install emcee
import emcee
import matplotlib.pyplot as plt
%matplotlib inline
import holoviews as hv
hv.extension('bokeh')

Output hidden; open in https://colab.research.google.com to view.

In [3]:
linewidths = 2
axislinewidths = 2
lenticks = 6

rc('font', family='sans-serif', size=18)
rc('xtick.major', size=lenticks)
rc('xtick.minor', size=lenticks * 2 / 3)
rc('ytick.major', size=lenticks)
rc('ytick.minor', size=lenticks * 2 / 3)
rc('lines', linewidth=linewidths)
rc('axes', linewidth=axislinewidths)


def votable_to_pandas(votable_file):
    votable = parse(votable_file)
    table = votable.get_first_table().to_table()
    return table.to_pandas()

### Preprocess

In [ ]:
table = votable_to_pandas('/content/drive/MyDrive/Be_Stars/ful_table.vot')

#bad astrometric solution
bad_RUWE = []
for i in range(0, len(table['name'])):
  if table['ruwe'][i]>1.4 :
    bad_RUWE.append(i)
bad_bes = table.iloc[bad_RUWE].reset_index()

#unrealistic distcances
bad_point = table[(1/table['parallax']<0)]
bad_points = table[1/table['parallax']>10]
spur = bad_points.append(bad_point)
bad_index = spur.index.to_list()
bad_distance = table.iloc[bad_index].reset_index()
bad_distance['name']

#multiple gaia crossmatches
duplicates_list = list(table['name'])
duplicates = sorted(set([i for i in duplicates_list if duplicates_list.count(i)>1]))
print(f'Number of multiple matches in DR3: {len(duplicates)}')
duplinds = []
for d in duplicates:
    duplinds.append([i for i,j in enumerate(duplicates_list) if j==d])
dupl_indices = list(chain.from_iterable(duplinds))
duplicate_bes = table.iloc[dupl_indices].reset_index()
duplicate_bes['name']

#final list of stars
drop_list = list(set( dupl_indices + bad_index))
newtable = table.drop(drop_list).reset_index()
print('Total', len(drop_list), 'stars are dropped')

Number of multiple matches in DR3: 14
Total 31 stars are dropped


In [5]:
newtable = pd.read_csv('/content/dupl.csv')

### pm correction

In [6]:
old_mu_alpha = newtable['pmra']  # gaia pmra comes with cos(declination)
old_mu_delta = newtable['pmdec']

''' 
magnitude dependent proper motion correction
'''

Wx = []
Wy = []
Wz = []
i = 0
while (i < len(newtable['parallax'])):
  if newtable['phot_g_mean_mag'][i]<9:
    Wx.append(18.4)
    Wy.append(33.8)
    Wz.append(-11.3)
  elif newtable['phot_g_mean_mag'][i]>9 and newtable['phot_g_mean_mag'][i]<9.5:
    Wx.append(14)
    Wy.append(30.7)
    Wz.append(-19.4)
  elif newtable['phot_g_mean_mag'][i]>9.5 and newtable['phot_g_mean_mag'][i]<10:
    Wx.append(12.8)
    Wy.append(31.4)
    Wz.append(-11.8)
  elif newtable['phot_g_mean_mag'][i]>10 and newtable['phot_g_mean_mag'][i]<10.5:
    Wx.append(13.6)
    Wy.append(35.7)
    Wz.append(-10.5)
  elif newtable['phot_g_mean_mag'][i]>10.5 and newtable['phot_g_mean_mag'][i]<11:
    Wx.append(16.2)
    Wy.append(50)
    Wz.append(2.1)
  elif newtable['phot_g_mean_mag'][i]>11 and newtable['phot_g_mean_mag'][i]<11.5:
    Wx.append(19.4)
    Wy.append(59.9)
    Wz.append(0.2)
  elif newtable['phot_g_mean_mag'][i]>11.5 and newtable['phot_g_mean_mag'][i]<11.75:
    Wx.append(21.8)
    Wy.append(64.2)
    Wz.append(1)
  elif newtable['phot_g_mean_mag'][i]>11.75 and newtable['phot_g_mean_mag'][i]<12:
    Wx.append(17.7)
    Wy.append(65.6)
    Wz.append(-1.9)
  elif newtable['phot_g_mean_mag'][i]>12 and newtable['phot_g_mean_mag'][i]<12.25:
    Wx.append(21.3)
    Wy.append(74.8)
    Wz.append(2.1)
  elif newtable['phot_g_mean_mag'][i]>12.25 and newtable['phot_g_mean_mag'][i]<12.5:
    Wx.append(25.7)
    Wy.append(73.6)
    Wz.append(1)
  elif newtable['phot_g_mean_mag'][i]>12.5 and newtable['phot_g_mean_mag'][i]<12.75:
    Wx.append(27.3)
    Wy.append(76.6)
    Wz.append(0.5)
  elif newtable['phot_g_mean_mag'][i]>12.75 and newtable['phot_g_mean_mag'][i]<13:
    Wx.append(34.9)
    Wy.append(68.9)
    Wz.append(-2.9)
  else:
    Wx.append(0)
    Wy.append(0)
    Wz.append(0)
  i = i+1  

In [7]:
dec = np.radians(newtable['dec'])
ra = np.radians(newtable['ra'])
mu_matrix = [[],[]]
zeros = np.zeros(len(newtable['ra']))
for i in range(0, len(Wz)):
  X = np.matmul([[-np.sin(dec)[i]*np.cos(ra)[i], -np.sin(dec)[i]*np.sin(ra)[i], np.cos(dec)[i]],[np.sin(ra)[i], -np.cos(ra)[i], 0]], [Wx[i], Wy[i], Wz[i]])
  mu_matrix[0].append(X[0]/1000)
  mu_matrix[1].append(X[1]/1000)
mu_alpha = old_mu_alpha - mu_matrix[0]
mu_delta = old_mu_delta - mu_matrix[1]

newtable['corrected_pmra'] = mu_alpha
newtable['corrected_pmdec'] = mu_delta

### distance velocity

In [8]:
import csv
data = ['star_name', 'vt_median', 'v68_lower', 'v68_upper', 'dist_median', 'dist68_lower', 'dist68_upper']
with open('/content/drive/MyDrive/thesis_final_results/distance_velocities_extra.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(data)
    f.close()

In [9]:
def logLikelihood(x, dist, vel, angle, invCov, Vmax):
    # Normal distribution
    c2 = 4.74047
    mu = np.array([1000/dist, 1000/dist/c2 * Vmax * vel * np.sin(angle), 1000/dist/c2 * Vmax * vel * np.cos(angle)])
    return -0.5 * np.dot(x-mu, np.dot(invCov, x-mu))
    
def logPriorDistance(d, L):
    # Gamma distribution
    return 2.0 * np.log(d) - d / L
    
def logPriorVelocity(v, vmax, alpha, beta):
    # Beta distribution
    return (alpha-1) * np.log(v/vmax) + (beta-1) * np.log(1-v/vmax)
    
def logPriorAngle(angle):
    # Uniform distribution
    twopi = 2.*np.pi
    if (angle > 0.0) and (angle <= twopi):
        return -np.log(twopi)
    else:
        return -np.inf
        
def logPosterior(theta, obs, invCov, Vmax, L):
    
    dist  = theta[0]
    vel   = theta[1]
    angle = theta[2]
    
    logPrior1 = logPriorDistance(dist, L)
    if not np.isfinite(logPrior1):
        return -np.inf
    
    logPrior2 = logPriorVelocity(vel, Vmax, 2, 3)
    if not np.isfinite(logPrior2):
        return -np.inf
    
    logPrior3 = logPriorAngle(angle)
    if not np.isfinite(logPrior3):
        return -np.inf
    
    logLike = logLikelihood(obs, dist, vel, angle, invCov, Vmax)
    logPost = logLike + logPrior1 + logPrior2 + logPrior3
    return  logPost

In [12]:
scaleLength = 500 #pc
Vmax = 500 #km/s
Ndim = 3
Nwalkers = 4 * Ndim
Nskip = 50

for i in range(0, len(newtable['parallax'])): #len(newtable['parallax'])):

    observed = np.array([newtable['parallax'][i], newtable['corrected_pmra'][i], newtable['corrected_pmdec'][i]])
    cov = np.zeros((3,3))
    cov[0,0] = newtable['parallax_error'][i]**2
    cov[1,1] = newtable['pmra_error'][i]**2
    cov[2,2] = newtable['pmdec_error'][i]**2
    cov[0,1] = newtable['parallax_pmra_corr'][i] * newtable['parallax_error'][i] * newtable['pmra_error'][i]
    cov[1,0] = cov[0,1]
    cov[0,2] = newtable['parallax_pmdec_corr'][i] * newtable['parallax_error'][i] * newtable['pmdec_error'][i]
    cov[2,0] = cov[0,2]
    cov[1,2] = newtable['pmra_pmdec_corr'][i] * newtable['pmra_error'][i] * newtable['pmdec_error'][i]
    cov[2,1] = cov[1,2]
    
    invCov = np.linalg.inv(cov)
    sampler = emcee.EnsembleSampler(Nwalkers, Ndim, logPosterior, args=(observed, invCov, Vmax, scaleLength))
    pos = [np.array([500., 50., 3.14]) + np.random.normal(size=Ndim) for n in range(Nwalkers)]
    output = sampler.run_mcmc(pos, 5000)
    samples = sampler.chain[:, Nskip:, :].reshape((-1, Ndim))
    unitFactors = np.array([1, Vmax, 180./np.pi])
    percentiles = np.percentile(samples, [16, 50, 84], axis=0) * unitFactors
    percentiles = np.round(percentiles, 3)
    
    star_data = [ newtable['name'][i], percentiles[1,0], percentiles[0,0], percentiles[2,0], percentiles[1,1], percentiles[0,1], percentiles[2,1] ]
    
    print(i)
    
    from csv import writer
    with open('/content/drive/MyDrive/thesis_final_results/distance_velocities_extra.csv', 'a') as f_object:
        writer_object = writer(f_object)
        writer_object.writerow(star_data)
        f_object.close()
    

0
1
2
3
